In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import random_split, DataLoader
import torchvision.datasets as datasets
import art.attacks.evasion as toolbox
from art.estimators.classification import PyTorchClassifier
import tqdm

In [ ]:
print("Check current device: ")
# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): # Should return True
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # Should show your GPU name
else:
    print("Using CPU")

### set RNG seed
RNG_SEED = 0

np.random.seed(RNG_SEED)
torch.manual_seed(RNG_SEED)
if torch.cuda.is_available():
	torch.cuda.manual_seed(RNG_SEED)
	torch.cuda.manual_seed_all(RNG_SEED)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

test_image = np.array([data[0].numpy() for data in test_dataset])
test_label = np.array([data[1] for data in test_dataset])

In [ ]:
eps_res = 10
eps_min = 0.0001
eps_max = 0.3

# Generating the list of values
eps_list = [eps_min + i * (eps_max - eps_min) / (eps_res - 1) for i in range(eps_res)]
noise_percents = np.arange(0.0, 0.55, 0.05)  # Noise levels from 0.0 to 0.5

files = ['MobileNetV3_large_CE.pth', 'MobileNetV3_small_10_CE.pth', 
         'KD_model_2_0.pth', 'KD_model_2_1.pth', 'KD_model_2_2.pth', 'KD_model_2_3.pth', 'KD_model_2_4.pth', 
         'KD_model_3_5.pth', 'KD_model_3_6.pth', 'KD_model_3_7.pth', 'KD_model_3_8.pth', 'KD_model_3_9.pth',
         'KD_model_4_10.pth', 'KD_model_4_11.pth', 'KD_model_4_12.pth', 'KD_model_4_13.pth', 'KD_model_4_14.pth',
         'KD_model_5_15.pth', 'KD_model_5_16.pth', 'KD_model_5_17.pth', 'KD_model_5_18.pth', 'KD_model_5_19.pth']

pgd_all_accuracies = []
snp_all_accuracies = []

In [ ]:
def toolbox_generate(eps, classifier, test_image=test_image, test_label=test_label):
    pgd_attack = toolbox.ProjectedGradientDescent(estimator=classifier, eps=eps)
    x_test_adv_pgd = pgd_attack.generate(x=test_image)
    predictions = classifier.predict(x_test_adv_pgd)
    accuracy = np.sum(np.argmax(predictions, axis=1) == test_label) / len(test_label)
    return accuracy

In [ ]:
# Salt-and-Pepper Noise Class (same as corrected version)
class SaltAndPepperNoise:
    def __init__(self, noise_percent=0.1):
        self.noise_percent = noise_percent

    def generate(self, xs):
        noisy_xs = np.copy(xs)
        num_pixels = int(self.noise_percent * xs.size)

        # Salt noise
        salt_coords = [np.random.randint(0, i, num_pixels // 2) for i in xs.shape]
        noisy_xs[tuple(salt_coords)] = 1.0

        # Pepper noise
        pepper_coords = [np.random.randint(0, i, num_pixels // 2) for i in xs.shape]
        noisy_xs[tuple(pepper_coords)] = 0.0

        return noisy_xs

In [ ]:
# Define loss function and optimizer
for f in files:  
    if f == 'MobileNetV3_large_CE.pth':
      torch.manual_seed(42)
      model = models.mobilenet_v3_large(weights=None)
    else:
      torch.manual_seed(42)
      model = models.mobilenet_v3_small(weights=None)

    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
    model.load_state_dict(torch.load(f, map_location=device, weights_only=True))
    model.eval()
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    classifier_pgd = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=optimizer,
        input_shape=(3, 224, 224), 
        nb_classes=10   
    )

    classifier_snp = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=optimizer,
        input_shape=(3, 224, 224), 
        nb_classes=10   
    )

    pgd_accuracy = []
    snp_accuracy = []

    print("--------------------------")
    print(f"Running PGD......... {f}")
    for eps in eps_list:
        accuracy = toolbox_generate(eps, classifier_pgd)
        pgd_accuracy.append(accuracy * 100)
        print(f"Accuracy with epsilon {eps}: {accuracy * 100:.2f}%")
    pgd_all_accuracies.append(pgd_accuracy)

    print(f"Running SNP......... {f}")

    # Apply Salt-and-Pepper Noise for each noise level
    for noise_percent in noise_percents:
        print(f"\nEvaluating Salt-and-Pepper Noise with noise_percent = {noise_percent}")
        spn_attack = SaltAndPepperNoise(noise_percent=noise_percent)

        # Initialize lists to collect noisy images and labels
        noisy_test_images = []
        test_labels = []

        # Generate noisy images for the entire test set
        for images, labels in tqdm(test_loader, desc=f"Adding Noise (percent={noise_percent})"):
            noisy_images = spn_attack.generate(images.numpy())
            noisy_test_images.append(noisy_images)
            test_labels.append(labels.numpy())

        # Concatenate all noisy examples and labels
        noisy_test_images = np.vstack(noisy_test_images)
        test_labels = np.hstack(test_labels)

        # Predict on noisy examples
        preds = np.argmax(classifier_snp.predict(noisy_test_images), axis=1)

        # Calculate accuracy
        accuracy = np.mean(preds == test_labels) * 100
        print(f"Accuracy with noise_percent={noise_percent}: {accuracy:.2f}%")

        # Store the results
        snp_accuracy.append(accuracy)
    snp_all_accuracies.append(snp_accuracy)
    

In [ ]:
print(pgd_all_accuracies)

In [ ]:
print(snp_all_accuracies)